##### *This notebook provides a full example on how to build a firmware for Machine Learning (Deep Learning) Inference on FPGA through the BondMachine framework. The are two main steps: in the first one, the user has to train a model and create a BondMachine Deep Learning firmware through pybondmachine, a library developed to abstract the BondMachine framework. In the second step, the user will use the firmware to perform inference on FPGA.*

In [ ]:
!pip install tensorflow==2.11.0
!pip install --upgrade pybondmachine
!pip install typing-extensions --upgrade
!pip install pandas

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### **Env variables**

- **`BONDMACHINE_DIR`** - the directory where bondmachine tools are installed
- **`XILINX_HLS`** - the directory where Xilinx HLS is installed
- **`XILINX_VIVADO`** - the directory where Xilinx Vivado is installed
- **`XILINX_VITIS`** - the directory where Xilinx Vitis is installed
- **`PATH`** - add the bondmachine tools to the path


In [ ]:
import os
BONDMACHINE_DIR="/home/"+os.environ["USER"]+"/bin"
os.environ["BONDMACHINE_DIR"]=BONDMACHINE_DIR
os.environ["PATH"]=os.environ["PATH"]+":"+os.environ["BONDMACHINE_DIR"]
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2'
os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2023.2'
os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2023.2'
os.environ['PATH']=os.environ["PATH"]+":"+os.environ['XILINX_HLS']+"/bin:"+os.environ['XILINX_VIVADO']+"/bin:"+os.environ['XILINX_VITIS']+"/bin:"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#### **Libraries**

- **`os`** - provides a portable way of using operating system dependent functionality
- **`sklearn`** - provides simple and efficient tools for predictive data analysis
- **`numpy`** - provides a fast numerical array structure and helper functions
- **`csv`** - provides classes for reading and writing tabular data in CSV format
- **`matplotlib`** - provides a MATLAB-like plotting framework
- **`tensorflow`** - provides an open source machine learning framework

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import json
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam, Adagrad
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt

#### **Data**

##### Download the dataset from openML. OpenML is a platform for sharing machine learning data, tasks and experiments. It's a great place to find datasets to work with. You can find the dataset we are going to use here: https://www.openml.org/d/42468

In [ ]:
dataset = "banknote-authentication"
data = fetch_openml(dataset)
x_data, y_data = data['data'], data['target']

##### **Process data**

##### Process the data, split the data in train and tast sample and save them for later use. The test sample will be used to test the model on the hardware and to compare the results with the model running on FPGA

In [ ]:
le = LabelEncoder() # create a new instance of label encoder class, which is used to encode categorical labels as numerical values
y = le.fit_transform(y_data) # this methods fits the encoder to the unique values in y_data and then transforms y_data into an array of encoded values
unique = np.unique(y) # get the unique values in y
y = to_categorical(y, len(unique)) # convert the encoded y array into a one-hot encoded matrix. len(unique) is the number of classes in the dataset

X_train_val, X_test, y_train_val, y_test = train_test_split(x_data, y, test_size=0.2, random_state=42)

# perform feature scaling on the training and testing data using the StandardScaler class from sklearn.preprocessing module.
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val) # fit the scaler to the training data and then transform it
X_test = scaler.transform(X_test) # transform the test data using the scaler that was fit to the training data
classes = le.classes_
classes_len = len(classes)

if not os.path.exists('datasets'):
    os.makedirs('datasets')

np.save("datasets/"+dataset+'_X_train_val.npy', X_train_val)
np.save("datasets/"+dataset+'_X_test.npy', X_test)
np.save("datasets/"+dataset+'_y_train_val.npy', y_train_val)
np.save("datasets/"+dataset+'_y_test.npy', y_test)
np.save("datasets/"+dataset+'_classes.npy', le.classes_)

with open("datasets/"+dataset+'_sample.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(X_test)

In [ ]:
!ls datasets

#### **Neural Network model**

##### Build the model starting from a JSON file that describes the network. The network architecture as well as the training process is not the main focus of this notebook, the complexity of the network is not important and it is very simple. The main focus is to show how to use the bondmachine tools to accelerate the inference of a neural network on FPGA. #####

In [ ]:
!cat nn-specifics.json

In [ ]:
train_size = X_train_val.shape[0]
model = Sequential()
f = open('nn-specifics.json')
network_spec = json.load(f)
f.close()

arch = network_spec["network"]["arch"]
for i in range(0, len(arch)):
    layer_name = network_spec["network"]["arch"][i]["layer_name"]
    activation_function = network_spec["network"]["arch"][i]["activation_function"]
    neurons = network_spec["network"]["arch"][i]["neurons"]
    if i == 0:
        model.add(Dense(neurons, input_shape=(X_train_val.shape[1],), kernel_regularizer=l1(0.0001)))
    else:
        model.add(Dense(neurons, activation=activation_function, name=layer_name, kernel_regularizer=l1(0.0001)))

if  network_spec["network"]["training"]["optimizer"] == "Adam":
    opt = Adam(learning_rate=0.0001)
elif network_spec["network"]["training"]["optimizer"] == "Adagrad":
    opt = Adagrad(learning_rate=0.0001)
else:
    opt = Adam(learning_rate=0.0001)

model.add(Dense(classes_len, activation='softmax'))
model.compile(optimizer=opt, loss=['categorical_crossentropy'], metrics=['accuracy'])

#### **TRAINING**

#### Train the model and save the predictions for later use

In [ ]:
# In this block of code we train the neural network and save the model.
checkpoint_path = 'models/'+dataset+'/training/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True)
batch_size = int(X_train_val.shape[1]*10) if network_spec["network"]["training"]["batch_size"] == "default" else int(network_spec["network"]["training"]["batch_size"])
epochs = int(network_spec["network"]["training"]["epochs"])
validation_split = 0.25 if network_spec["network"]["training"]["validation_split"] == "default" else float(network_spec["network"]["training"]["validation_split"])
shuffle = True if network_spec["network"]["training"]["shuffle"] == "true" else False

model.fit(X_train_val, y_train_val, batch_size=batch_size, epochs=epochs, validation_split=validation_split, shuffle=shuffle, callbacks=[cp_callback], verbose=0)

test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Dump the keras predictions of the test sample to be used as cross validation for the BondMachine NN
y_keras = model.predict(X_test)
np.save("datasets/"+dataset+'_y_keras.npy', y_keras)

results = [[*pred, np.argmax(pred)] for pred in y_keras]
fields = ['probability_' + str(i) for i in range(len(classes))] + ['classification']

with open("datasets/"+dataset+'_sw.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(results)

In [ ]:
!ls models/banknote-authentication/training

#### **BONDMACHINE**

##### Use the `pybondmachine` library to create, setup and build the firmware. Start from the basic import #####

In [ ]:
from pybondmachine.prjmanager.prjhandler import BMProjectHandler
from pybondmachine.converters.tensorflow2bm import mlp_tf2bm

##### Convert the trained neural network model into a standard file intepretable by `neuralbond`, the BondMachine tool that will convert the model into a set of heterogeneus connecting processors that represent the neural network #####

In [ ]:
output_file = "modelBM.json"
output_path = os.getcwd()+"/output/"

mlp_tf2bm(model, output_file=output_file, output_path=output_path)

##### Create the `BondMachine` object, this object will be used to create the project, setup configuration and build the firmeare #####

In [ ]:
prjHandler = BMProjectHandler("sample_project", "neuralnetwork", "projects_tests")
prjHandler.check_dependencies() # bmhelper

In [ ]:
prjHandler.create_project(target_board='alveou50')

##### Define the configuration of the project, in details:
- **`data_type`** - the data type of the bondmachine architecture
- **`register_size`** - the size of the register in the bondmachine architecture
- **`source_neuralbond`** - the path to the neuralbond source file
- **`flavor`** - the interconnection protocol used to interact with the firmware
- **`board`** - the target device where the firmware will run

##### and then setup the configuration of the project #####


In [ ]:
config = {
    "data_type": "float16",
    "register_size": "16",
    "source_neuralbond": output_path+output_file,
    "flavor": "axist",
    "board": "alveou50"
}

prjHandler.setup_project(config)

##### Finally, build the firmware. The `oncloud` parameter is used to specify if the firmware will be built on the cloud or locally. Build the firmware using this library is not actually supported, but we have developed a prototype of Inference As a Service System that includes the build firmware process on a remote host #####

In [ ]:
# This will take more than an 1 hour, some firmwares have already been prepared
#prjHandler.build_firmware(oncloud=False)

In [ ]:
os.environ['XILINX_XRT'] = '/opt/xilinx/xrt'
os.environ['LD_LIBRARY_PATH'] = '/opt/xilinx/xrt/lib'

notebook_directory = os.path.abspath(os.path.dirname((os.environ["JPY_SESSION_NAME"])))
os.chdir(notebook_directory)

In [ ]:
# Download the bitstream file from bondmachine.fisica.unipg.it
!wget -nc http://bondmachine.fisica.unipg.it/firmwares/bmfloat16.xclbin

In [ ]:
from pybondmachine.overlay.predictor import Predictor

In [ ]:
import numpy as np
model_specs = {
    "data_type": "float16",
    "register_size": 16,
    "batch_size": 16,
    "flavor": "axist",
    "n_input": 4,
    "n_output": 2,
    "benchcore": True,
    "board": "alveo"
}
firmware_name = "bmfloat16.xclbin"
firmware_path = ""

In [ ]:
X_test = np.load("datasets/banknote-authentication_X_test.npy")
y_test = np.load("datasets/banknote-authentication_y_test.npy")

In [ ]:
predictor = Predictor(firmware_name, firmware_path, model_specs)

In [ ]:
predictor.load_overlay()

In [ ]:
predictor.prepare_data(X_test, y_test)

In [ ]:
status, result = predictor.predict(debug=True)

In [ ]:
print(result)